![CoSAppLogo](images/cosapp.svg) **CoSApp** tutorials:

# Drivers

## What is a `Driver`?!

`Driver` objects allow users to modify the state of a `System` for a given purpose. Resolution of non-linear equations, optimization, or time integration of dynamic equations are typical examples of `Driver` usage.
As will be seen in this tutorial, drivers can also be nested, to create a simulation workflow.

## Introduction

### Add a `Driver`

Simply use the `add_driver` method passing the `Driver` object you want to use 
(see [Available drivers](#Available-Drivers) section of this tutorial).

In [ ]:
from cosapp.tests.library.systems import Multiply1
from cosapp.drivers import RunOnce

m = Multiply1('mult')
run = m.add_driver(RunOnce('run'))

`Multiply1` *is the same* `System` *as* `Multiply` *in the* [Systems](01-Systems.ipynb) *tutorial*

 ![Driver in system](images/drivers_1.svg)

### Implementation

Every `System` (including sub-systems) may have one or multiple `Driver` objects. They are stored in the `drivers` attribute.
By default, there is no `Driver` attached to a `System`.

The `run_drivers` method of `System` executes drivers recursively. In the following example, the simplest driver
[RunOnce](#RunOnce) is added and executed.

In [ ]:
from cosapp.tests.library.systems import Multiply1

m = Multiply1('mult')
m.add_driver(RunOnce('run'))
print('m.drivers:', m.drivers) # print drivers of the system

m.K1 = 2.
m.p_in.x = 15.
m.run_drivers()

print(f"m.p_out.x = {m.p_out.x}")

### Driver chains and subdrivers

If several drivers are attached to a system, `run_drivers` will execute them in turn, as a sequence of drivers.
Furthermore, like a `System`, each individual `Driver` may have children, which also inherit from base class `Driver`. Nested drivers are created with method `add_child` of class `Driver`; they are stored in attribute `children` of the parent driver.

By construction, a `System` can have as many levels of drivers as required.

Driver chains and nested drivers thus allow users to define complex simulation scenarios, such as workflows, multi-point design, designs of experiment, optimization, *etc.*


![drivers](images/drivers_2.svg)


In [ ]:
m = Multiply1('mult')
run = m.add_driver(RunOnce('run'))

print(f"run.children: {run.children}")  # driver 'run' has no child

In [ ]:
subrun = run.add_child(RunOnce('subrun'))  # add a sub-driver 'subrun'

print(f"run.children: {run.children}", f"subrun.children: {subrun.children}", sep="\n")

## Available Drivers

**CoSApp** comes with a set of drivers to help users build their simulations.

### RunOnce

As the name suggests, `RunOnce` makes your `System` and its subsystems compute their code once. It does not deal with residues or iterative loops that may be necessary to resolve the `System`. Instead, it merely transports information from the top system down to the lowest level sub-systems.

In [ ]:
from cosapp.systems import System
from cosapp.tests.library.ports import XPort

class MultiplyWithResidue(System):

    def setup(self):
        self.add_input(XPort, 'p_in', {'x': 1.})
        self.add_output(XPort, 'p_out', {'x': 1.})

        self.add_inward('K1', 5.)
        
        # off design default
        self.add_inward('expected_output', 7.5)
        self.add_unknown('p_in.x').add_equation('expected_output == p_out.x')

        
    def compute(self):
        self.p_out.x = self.p_in.x * self.K1


In [ ]:
from cosapp.drivers import RunOnce

m = MultiplyWithResidue('mult')
run = m.add_driver(RunOnce('run'))

m.run_drivers()

print("List of defined drivers\n", dict(m.drivers))

print("",
    f"K1 = {m.K1}",
    f"p_in.x = {m.p_in.x}",
    f"p_out.x = {m.p_out.x}",
    f"residues: {list(m.residues.values())}",
    sep="\n"
)

### NonLinearSolver

This `Driver` determines the parameters of your `System` declared as unknowns that satisfy its equations. It resolves the mathematical problem between free parameters and residues of its child drivers.

NonLinear options are available:

- *tol* - solver iterates until *residue <= tol* (1e-5 by default).
- *factor* - relaxation factor applied at each iteration (1 by default; must be striclty positive).
- *max_iter* - maximum number of iterations (500 by default).

We define below a new `System` containing an equation, and execute a `NonLinearSolver` driver.

An `unknown` is simply defined by its name. An `equation` is defined by a string of the kind `"lhs == rhs"`, where `lhs` and `rhs` denote the left- and right-hand sides. Each side of the `equation` may be a constant or an evaluable expression, as in `"x - cos(sub.y) == z**2"`, or  `"x == 1.5"`. An exception will be raised if both sides are trivially constant, as in `"pi / 2 == 0"`.

In [ ]:
from cosapp.systems import System
from cosapp.tests.library.ports import XPort

class MultiplyWithResidue(System):

    def setup(self):
        self.add_input(XPort, 'p_in', {'x': 1.})
        self.add_output(XPort, 'p_out', {'x': 1.})

        self.add_inward('K1', 5.)
        
        # off design default
        self.add_inward('expected_output', 7.5)
        self.add_unknown('p_in.x').add_equation('p_out.x == expected_output')

        
    def compute(self):
        self.p_out.x = self.p_in.x * self.K1

In [ ]:
from cosapp.drivers import NonLinearSolver

m = MultiplyWithResidue('mult')
solver = m.add_driver(NonLinearSolver('solver', max_iter=100, tol=1e-3, factor=0.1))
m.run_drivers()

print("m.drivers:", m.drivers)
print(m.drivers['solver'].children)
print("",
    f"K1 = {m.K1}",
    f"p_in.x = {m.p_in.x}",
    f"p_out.x = {m.p_out.x}",
    f"m.expected_output = {m.expected_output}",
    f"residues: {list(m.residues.values())}",
    sep="\n"
)

### RunSingleCase

`NonLinearSolver` is primarily designed to solve [multi-point problems](./08-Multipoints-Design.ipynb), where several [RunSingleCase](#RunSingleCase) drivers are declared as direct sub-drivers of the solver.
By default, a `NonLinearSolver` driver comes with one [RunSingleCase](#RunSingleCase) child, called *runner*. 

![scipysolver](images/drivers_nonlinear.svg)

`RunSingleCase` executes all subsystem drivers by recursively calling the `compute()` method of the top system and each of its children.

This `Driver` does not contain a solver per se, but is helpfull to set boundary conditions, initial values, and define additional unknowns and/or equations.
It is primarily meant to be used as a child of a [NonLinearSolver](#NonLinearSolver) driver.

To update its `System` owner, two possibilities are offered:

- `set_values` will impose the value of prescribed input variables, as boundary conditions;
- `set_init` will change the initial value of iteratives before resolving the case.

Both methods take as argument a dictionary of the kind `{varname: value, ...}`, where `varname` is the name of the variable *in the context of the driver owner*.
For example, if the driver is attached to a system `head` possessing a child named `sub` and an inward `x`, `{'sub.k': 0.0, 'x': 0.1}` will affect variables `head.sub.k` and `head.x`.

In [ ]:
from cosapp.drivers import RunSingleCase

m = MultiplyWithResidue('mult')
update = m.add_driver(RunSingleCase('update'))
update.offdesign.add_unknown('p_in.x')

update.set_values({'expected_output': 15, 'K1': 2})
update.set_init({'p_in.x': 1.5})
m.run_drivers()

print("List of defined drivers:", m.drivers, sep="\n")
print("",
    f"K1 = {m.K1}",
    f"p_in.x = {m.p_in.x}",
    f"p_out.x = {m.p_out.x}",
    f"residues: {list(m.residues.values())}",
    sep="\n"
)

Another important ability of this driver is the addition of unknowns and equations to the mathematical system.
There are two kinds of problems for a `System`:

- *Design problems*: They are associated with a design variable that is frozen in the final product; e.g. the diameter of a pipe.
This type of unknowns and equation can be added to attribute `design`.
- *Off-design problems*: They correspond to constraints imposed regardless of design constraints (by physical laws, typically).
This type of unknowns and equation can be added to attribute `offdesign`.

The methods to call are identical:

```python
runcase.design \
       .add_unknown('pipe.diameter') \  # Name of unknown variable
       .add_equation('pipe.flow_in.W == 50')  # Constraint equation

runcase.offdesign \
       .add_unknown('pedal.angle') \  # Name of unknown variable
       .add_equation('outlet.pressure == p_atm')  # Constraint equation
```

The good practice is to have by default the system operating in off-design condition, and introduce design methods to design it (see [tutorial on design methods](11-DesignMethods.ipynb)).

It can happen that the system has some degrees of freedom freed without constraint equations
defined. So it is up to the user to feed the mathematical system with the proper degree of freedom and constraints to close it.

The solver will check if the mathematical system is closed before solving it.

In [ ]:
m.drivers.clear() # Remove all drivers on the system `m`
solver = m.add_driver(NonLinearSolver('solver'))
update = solver.add_child(RunSingleCase('update'))

# Customization of the case
update.set_values({'expected_output': 15.})

# Execution
m.run_drivers()

print(
    "Off-design default",
    f"K1 = {m.K1}",
    f"p_in.x = {m.p_in.x}",
    f"p_out.x = {m.p_out.x}",
    sep="\n"
)

Design methods could be defined at system level and be available for simulation.

In [ ]:
class MultiplyWithDesignMethod(System):

    def setup(self):
        self.add_input(XPort, 'p_in', {'x': 1.})
        self.add_output(XPort, 'p_out', {'x': 1.})

        self.add_inward('K1', 5.)
        
        # off design default
        self.add_inward('expected_output', 7.5)
        self.add_unknown('p_in.x').add_equation('p_out.x == expected_output')

        # design methods
        self.add_inward('dx_design', 1.0)        
        self.add_design_method('dx').add_unknown('K1').add_equation('p_out.x - p_in.x == dx_design')

    def compute(self):
        self.p_out.x = self.p_in.x * self.K1


from cosapp.recorders import DataFrameRecorder

m = MultiplyWithDesignMethod('m')
m.expected_output = 7.5

solver = m.add_driver(NonLinearSolver('solver', tol=1e-12))
solver.add_recorder(DataFrameRecorder(includes=['K1', 'p_*.x', 'expected*']))

# Design
design = solver.add_child(RunSingleCase('design'))
design.set_values({
    'expected_output': 7.5,
    'dx_design': 5,
})
design.design.extend(m.design_methods['dx'])

m.run_drivers()

print(
    "Design:",
    f"K1 = {m.K1}",
    f"p_in.x = {m.p_in.x}",
    f"p_out.x = {m.p_out.x}",
    sep="\n"
)
print(solver.problem)

With driver RunSingleCase, it is possible to simulate a life cycle made of states. On next exemple, design is made than offdesign for several environement conditions. 

![metadriver](images/drivers_5.svg)

In [ ]:
# Off-design 1
offdesign1 = solver.add_child(RunSingleCase('offdesign1'))
offdesign1.set_values({'expected_output': 15.})

# Off-design 2
offdesign2 = solver.add_child(RunSingleCase('offdesign2'))
offdesign2.set_values({'expected_output': 24.})

m.run_drivers()

df = solver.recorder.export_data()  # export recorded data as a pandas DataFrame
df

For advanced design methods tutorial and strategy to create a "design model" from a "simulation model", we recommand to read [Design Methods](./11-DesignMethods.ipynb) tutorial, before 
creating your test case.

**Congrats!** You are now ready to launch computation on your `System` with **CoSApp**!